In [2]:
import numpy as np
import pandas as pd
from ast import literal_eval
from pathlib import Path
from pprint import pprint
from sympy.combinatorics import Permutation
import matplotlib.pyplot as plt

# local data_dir
data_dir = Path("data")

puzzle_info = pd.read_csv(data_dir / 'puzzle_info.csv', index_col='puzzle_type')
# Parse allowed_moves
puzzle_info['allowed_moves'] = puzzle_info['allowed_moves'].apply(literal_eval)
# puzzle_info_df['total_allowed_moves'] = puzzle_info_df['allowed_moves_dict'].apply(lambda x: len(x.keys()))
puzzle_info['number_moves'] = puzzle_info['allowed_moves'].apply(lambda x: len(x.keys()))

puzzles_all = pd.read_csv(data_dir / 'puzzles.csv', index_col='id')
# Parse color states
puzzles_all = puzzles_all.assign(
    initial_state=lambda df: df['initial_state'].str.split(';'),
    solution_state=lambda df: df['solution_state'].str.split(';')
)

puzzles_all['total_components'] = puzzles_all['solution_state'].apply(len)
puzzles_all['all_unique_components'] = puzzles_all['solution_state'].apply(lambda x: np.unique(x))
puzzles_all['unique_components'] = puzzles_all['all_unique_components'].apply(len)

# load the sample_submission
ss = pd.read_csv(data_dir / 'sample_submission.csv', index_col='id')
ss['moves'] = ss['moves'].str.split('.')
ss['move_count'] = ss['moves'].apply(len)

In [5]:
np.array(puzzles_all.loc[32]['solution_state'])

array(['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C',
       'C', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'E', 'E', 'E',
       'E', 'E', 'E', 'E', 'E', 'E', 'F', 'F', 'F', 'F', 'F', 'F', 'F',
       'F', 'F'], dtype='<U1')

In [7]:
# create a set of all unique solutions for each puzzle type
unique_solutions = {}
for puzzle_type in puzzle_info.index:
    unique_solutions[puzzle_type] = set(puzzles_all.loc[puzzles_all['puzzle_type'] == puzzle_type, 
                                                        'solution_state'].apply(tuple).unique())

# print the unique solutions for cube_3/3/3 with each solution on a new line and the set as a list
print('\n'.join(map(str, unique_solutions['cube_3/3/3'])))



('N0', 'N1', 'N2', 'N3', 'N4', 'N5', 'N6', 'N7', 'N8', 'N9', 'N10', 'N11', 'N12', 'N13', 'N14', 'N15', 'N16', 'N17', 'N18', 'N19', 'N20', 'N21', 'N22', 'N23', 'N24', 'N25', 'N26', 'N27', 'N28', 'N29', 'N30', 'N31', 'N32', 'N33', 'N34', 'N35', 'N36', 'N37', 'N38', 'N39', 'N40', 'N41', 'N42', 'N43', 'N44', 'N45', 'N46', 'N47', 'N48', 'N49', 'N50', 'N51', 'N52', 'N53')
('A', 'B', 'A', 'B', 'A', 'B', 'A', 'B', 'A', 'B', 'C', 'B', 'C', 'B', 'C', 'B', 'C', 'B', 'C', 'D', 'C', 'D', 'C', 'D', 'C', 'D', 'C', 'D', 'E', 'D', 'E', 'D', 'E', 'D', 'E', 'D', 'E', 'F', 'E', 'F', 'E', 'F', 'E', 'F', 'E', 'F', 'A', 'F', 'A', 'F', 'A', 'F', 'A', 'F')
('A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F')


In [28]:
# how many unique solutions for cube_3/3/3
print(len(unique_solutions['cube_3/3/3']))

# create dictionary with keys = puzzle_type and values = length of unique solutions
unique_solutions_count = {k: len(v) for k, v in unique_solutions.items()}

unique_solutions_df = pd.DataFrame.from_dict(unique_solutions_count, orient='index', columns=['unique_solutions'])

# add a column to unique_solutions_df with the number of puzzles in each puzzle_type
unique_solutions_df['puzzle_count'] = 0
for type in puzzles_all['puzzle_type'].unique():
    unique_solutions_df.loc[type, 'puzzle_count'] = len(puzzles_all.loc[puzzles_all['puzzle_type'] == type])

unique_solutions_df.head(20)


3


,unique_solutions,puzzle_count
cube_2/2/2,3,30
cube_3/3/3,3,120
cube_4/4/4,3,60
cube_5/5/5,3,35
cube_6/6/6,3,12
cube_7/7/7,1,5
cube_8/8/8,1,5
cube_9/9/9,1,5
cube_10/10/10,1,5
cube_19/19/19,1,4


In [27]:
print(len(puzzles_all[puzzles_all.puzzle_type == 'cube_3/3/3']))

for type in puzzles_all.puzzle_type.unique():
    print(type, len(puzzles_all[puzzles_all.puzzle_type == type]))

temp = [(type, len(puzzles_all[puzzles_all.puzzle_type == type])) for type in puzzles_all.puzzle_type.unique()]
print(temp)

120
cube_2/2/2 30
cube_3/3/3 120
cube_4/4/4 60
cube_5/5/5 35
cube_6/6/6 12
cube_7/7/7 5
cube_8/8/8 5
cube_9/9/9 5
cube_10/10/10 5
cube_19/19/19 4
cube_33/33/33 3
wreath_6/6 20
wreath_7/7 15
wreath_12/12 10
wreath_21/21 5
wreath_33/33 3
wreath_100/100 1
globe_1/8 10
globe_1/16 5
globe_2/6 5
globe_3/4 15
globe_6/4 5
globe_6/8 5
globe_6/10 5
globe_3/33 8
globe_8/25 2
[('cube_2/2/2', 30), ('cube_3/3/3', 120), ('cube_4/4/4', 60), ('cube_5/5/5', 35), ('cube_6/6/6', 12), ('cube_7/7/7', 5), ('cube_8/8/8', 5), ('cube_9/9/9', 5), ('cube_10/10/10', 5), ('cube_19/19/19', 4), ('cube_33/33/33', 3), ('wreath_6/6', 20), ('wreath_7/7', 15), ('wreath_12/12', 10), ('wreath_21/21', 5), ('wreath_33/33', 3), ('wreath_100/100', 1), ('globe_1/8', 10), ('globe_1/16', 5), ('globe_2/6', 5), ('globe_3/4', 15), ('globe_6/4', 5), ('globe_6/8', 5), ('globe_6/10', 5), ('globe_3/33', 8), ('globe_8/25', 2)]
